<a href="https://colab.research.google.com/github/carisa40/sql_commands/blob/master/read_write_sqlpool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# specifying the connection details

jdbcHostname = "sponsoredtelefonica.sql.azuresynapse.net"
jdbcDatabase = "telefonicapool"
username = "sqladmin_telefonica"
password = "telefon2020@sclwk"
jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2};user={3};password={4}".format(jdbcHostname, jdbcPort, jdbcDatabase, username, password)

In [ ]:
# creating a spark table from the database table inside the pool
remote_table = spark.read.format("jdbc") \
                         .option("url",jdbcUrl) \
                         .option("dbtable", "Final_DataFrame") \
                         .load()


In [ ]:
# this configuration must be set to enable one to access the spark table


sc._jsc.hadoopConfiguration().set(
  "fs.azure.account.key.sponsoredtelefonicalake.blob.core.windows.net",
  "61B7uSwOkgBBc3winOXC7tR7cNbMesJyxLJtLA7INzn6VeP/JCsIwP0x7PK3ZMlQ+yDL/mjv3eXxrZZO/tugmg==") # this is the access key for the data lake storage account


In [ ]:
# testing the process of writing to the database.

sample = [("kofi",10), ("Ama",20), ("kwame",30), ("adjoa",40)]

Columns = ["name","age"]
sample_df = spark.createDataFrame(data=sample, schema = Columns)
sample_df.printSchema()
sample_df.show(truncate=False)

root
-- name: string (nullable = true)
-- age: long (nullable = true)

+-----+---+
name |age|
+-----+---+
kofi |10 |
Ama |20 |
kwame|30 |
adjoa|40 |
+-----+---+

In [ ]:
# code for creating a csv on databricks

sample_df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/Sample.csv")

In [ ]:
# Writing a new table to the database

sample_df.write \
  .format("com.databricks.spark.sqldw") \
  .option("url", jdbcUrl) \
  .option("forwardSparkAzureStorageCredentials", "true") \
  .option("tempDir", "wasbs://root@sponsoredtelefonicalake.blob.core.windows.net/tesst") \
  .option("dbTable", "dbo.test") \
  .save()

In [ ]:
# testing an append feature
append_sample = [("esi",10), ("esinam",20)]

append_sample_df = spark.createDataFrame(data=append_sample, schema = Columns)
append_sample_df.printSchema()
append_sample_df.show(truncate=False)


root
-- name: string (nullable = true)
-- age: long (nullable = true)

+------+---+
name |age|
+------+---+
esi |10 |
esinam|20 |
+------+---+

In [ ]:
append_sample_df.write \
  .format("com.databricks.spark.sqldw") \
  .mode("append") \
  .option("url", jdbcUrl) \
  .option("forwardSparkAzureStorageCredentials", "true") \
  .option("tempDir", "wasbs://root@sponsoredtelefonicalake.blob.core.windows.net/tesst") \
  .option("dbTable", "dbo.test") \
  .save()